# SNLI dataset
https://nlp.stanford.edu/projects/snli/

Conll-u format, used by stanza parser
https://universaldependencies.org/format.html

In [ ]:
!wget https://nlp.stanford.edu/projects/snli/snli_1.0.zip
!pip install tqdm boto3 requests regex sentencepiece sacremoses
!pip install transformers
!pip install stanza

--2023-06-13 13:19:11--  https://nlp.stanford.edu/projects/snli/snli_1.0.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94550081 (90M) [application/zip]
Saving to: ‘snli_1.0.zip’

snli_1.0.zip        100%[===================>]  90.17M  25.4MB/s    in 4.4s    

2023-06-13 13:19:15 (20.6 MB/s) - ‘snli_1.0.zip’ saved [94550081/94550081]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from zipfile import ZipFile
import time
import stanza
import pandas as pd
import json


## unzip

In [ ]:



with ZipFile('snli_1.0.zip', 'r') as zip:
    zip.extractall()

print('Extraction complete.')

# with ZipFile('sick_test_annotated.zip', 'r') as zip:
#     zip.extractall()

# print('Extraction complete.')


Extraction complete.


## SNLI hard from .txt

In [ ]:
!wget https://nlp.stanford.edu/projects/snli/snli_1.0_test_hard.jsonl

--2023-02-08 15:37:26--  https://nlp.stanford.edu/projects/snli/snli_1.0_test_hard.jsonl
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3180795 (3.0M) [text/plain]
Saving to: ‘snli_1.0_test_hard.jsonl’

snli_1.0_test_hard. 100%[===================>]   3.03M  1.22MB/s    in 2.5s    

2023-02-08 15:37:29 (1.22 MB/s) - ‘snli_1.0_test_hard.jsonl’ saved [3180795/3180795]



In [ ]:

df_hard = pd.read_json('snli_1.0_test_hard.jsonl', lines=True)
df_hard = df_hard[['gold_label','sentence1','sentence2']]#2
df_hard=df_hard.dropna()#3
df_hard

ValueError: ignored

## Training split

In [ ]:
df_train = pd.read_csv('snli_1.0/snli_1.0_train.txt', sep='\t', on_bad_lines='skip')#1
# df_train = df_train.dropna(subset=['label2', 'label3','label4','label5'])
df_train = df_train[['gold_label','sentence1','sentence2']]#2
df_train=df_train.dropna()#3
#df_train = replace_all_special_tokens(df_train)#4
# df_train[df_train['sentence1'].str.contains('- ')]

In [ ]:
df_train['sentence1'].str.split().apply(len).mean()

12.850372446586906

In [ ]:
df_train['sentence2'].str.split().apply(len).mean()

7.416556695858917

In [ ]:
df_train['text_sentence1'] = df_train['sentence1'].apply(lambda x: x.split())


In [ ]:
df_train['text_sentence2'] = df_train['sentence2'].apply(lambda x: x.split())

In [ ]:

max_length1 = df_train['text_sentence1'].apply(len)
max_length2 = df_train['text_sentence2'].apply(len)

print("Max length both:", (max_length1+max_length2).max())
print("Max length premise:", (max_length1).max())


Max length both: 112
Max length premise: 78


## Validation split

In [ ]:
df_val = pd.read_csv('snli_1.0/snli_1.0_dev.txt', sep='\t', on_bad_lines='skip')#1
df_val = df_val[['gold_label','sentence1','sentence2']]#2
df_val=df_val.dropna()#3
# df_val=replace_all_special_tokens(df_val)#4

## Test split

In [ ]:
df_test = pd.read_csv('snli_1.0/snli_1.0_test.txt', sep='\t', on_bad_lines='skip')#1
df_test = df_test[['gold_label','sentence1','sentence2']]#2
df_test=df_test.dropna()#3
# df_test = replace_all_special_tokens(df_test)#4

In [ ]:
# df_test = pd.read_pickle('SNLI_test_original')#1
# df_test

#df_test.to_pickle('SNLI_test_original.pickle')

,gold_label,sentence1_binary_parse,sentence2_binary_parse,sentence1_parse,sentence2_parse,sentence1,sentence2,captionID,pairID,label1,label2,label3,label4,label5
0,neutral,( ( This ( church choir ) ) ( ( ( sings ( to (...,( ( The church ) ( ( has ( cracks ( in ( the c...,(ROOT (S (NP (DT This) (NN church) (NN choir))...,(ROOT (S (NP (DT The) (NN church)) (VP (VBZ ha...,This church choir sings to the masses as they ...,The church has cracks in the ceiling.,2677109430.jpg#1,2677109430.jpg#1r1n,neutral,contradiction,contradiction,neutral,neutral
1,entailment,( ( This ( church choir ) ) ( ( ( sings ( to (...,( ( The church ) ( ( is ( filled ( with song )...,(ROOT (S (NP (DT This) (NN church) (NN choir))...,(ROOT (S (NP (DT The) (NN church)) (VP (VBZ is...,This church choir sings to the masses as they ...,The church is filled with song.,2677109430.jpg#1,2677109430.jpg#1r1e,entailment,entailment,entailment,neutral,entailment
2,contradiction,( ( This ( church choir ) ) ( ( ( sings ( to (...,( ( ( A choir ) ( singing ( at ( a ( baseball ...,(ROOT (S (NP (DT This) (NN church) (NN choir))...,(ROOT (NP (NP (DT A) (NN choir)) (VP (VBG sing...,This church choir sings to the masses as they ...,A choir singing at a baseball game.,2677109430.jpg#1,2677109430.jpg#1r1c,contradiction,contradiction,contradiction,contradiction,contradiction
3,neutral,( ( ( A woman ) ( with ( ( ( ( ( a ( green hea...,( ( The woman ) ( ( is young ) . ) ),(ROOT (NP (NP (DT A) (NN woman)) (PP (IN with)...,(ROOT (S (NP (DT The) (NN woman)) (VP (VBZ is)...,"A woman with a green headscarf, blue shirt and...",The woman is young.,6160193920.jpg#4,6160193920.jpg#4r1n,neutral,neutral,neutral,neutral,neutral
4,entailment,( ( ( A woman ) ( with ( ( ( ( ( a ( green hea...,( ( The woman ) ( ( is ( very happy ) ) . ) ),(ROOT (NP (NP (DT A) (NN woman)) (PP (IN with)...,(ROOT (S (NP (DT The) (NN woman)) (VP (VBZ is)...,"A woman with a green headscarf, blue shirt and...",The woman is very happy.,6160193920.jpg#4,6160193920.jpg#4r1e,entailment,entailment,contradiction,entailment,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,contradiction,( ( Two women ) ( ( are ( ( observing somethin...,( ( ( Two women ) ( are ( standing ( with ( th...,(ROOT (S (NP (CD Two) (NNS women)) (VP (VBP ar...,(ROOT (S (NP (NP (CD Two) (NNS women)) (SBAR (...,Two women are observing something together.,Two women are standing with their eyes closed.,4378810163.jpg#4,4378810163.jpg#4r1c,contradiction,contradiction,contradiction,contradiction,contradiction
9996,entailment,( ( Two women ) ( ( are ( ( observing somethin...,( ( Two girls ) ( ( are ( looking ( at somethi...,(ROOT (S (NP (CD Two) (NNS women)) (VP (VBP ar...,(ROOT (S (NP (CD Two) (NNS girls)) (VP (VBP ar...,Two women are observing something together.,Two girls are looking at something.,4378810163.jpg#4,4378810163.jpg#4r1e,entailment,entailment,entailment,contradiction,entailment
9997,contradiction,( ( ( ( ( A man ) ( in ( a ( black ( leather j...,( ( A man ) ( ( is ( flying ( a kite ) ) ) . ) ),(ROOT (S (NP (NP (NP (DT A) (NN man)) (PP (IN ...,(ROOT (S (NP (DT A) (NN man)) (VP (VBZ is) (VP...,A man in a black leather jacket and a book in ...,A man is flying a kite.,152881593.jpg#1,152881593.jpg#1r1c,contradiction,contradiction,contradiction,contradiction,contradiction
9998,entailment,( ( ( ( ( A man ) ( in ( a ( black ( leather j...,( ( A man ) ( ( is ( speaking ( in ( a classro...,(ROOT (S (NP (NP (NP (DT A) (NN man)) (PP (IN ...,(ROOT (S (NP (DT A) (NN man)) (VP (VBZ is) (VP...,A man in a black leather jacket and a book in ...,A man is speaking in a classroom.,152881593.jpg#1,152881593.jpg#1r1e,entailment,entailment,entailment,neutral,entailment


## val hard from distilbert

In [ ]:
df_val_hard = pd.read_pickle('SNLI_val_hard.pickle')
df_val_hard = df_val_hard[['gold_label','sentence1','sentence2']]#2
df_val_hard=df_val_hard.dropna()#3
df_val_hard

,gold_label,sentence1,sentence2
3,entailment,"Two young children in blue jerseys, one with t...",Two kids in numbered jerseys wash their hands.
4,neutral,"Two young children in blue jerseys, one with t...",Two kids at a ballgame wash their hands.
5,contradiction,"Two young children in blue jerseys, one with t...",Two kids in jackets walk to school.
8,entailment,A man selling donuts to a customer during a wo...,A man selling donuts to a customer.
9,entailment,Two young boys of opposing teams play football...,boys play football
...,...,...,...
9984,entailment,Two police officers are sitting on motorcycles...,the police are sitting
9985,contradiction,A man is performing an aerial skateboard trick...,A man is ordering a coffee from a barista at a...
9989,contradiction,A man is performing an aerial skateboard trick...,He does a sick bicycle trick off the ramp
9992,entailment,A uniformed competitor in motocross has crosse...,A man is performing motorcycle jump across the...


# HANS (after processing one time. If using original dataset, columns will be named differently)

In [ ]:
df_val_hans = pd.read_pickle('HANS_val.pickle')
df_val_hans = df_val_hans[['gold_label','sentence1','sentence2']]#2
df_val_hans=df_val_hans.dropna()#3
df_val_hans

,gold_label,sentence1,sentence2
0,1,The president advised the doctor .,The doctor advised the president .
1,1,The student saw the managers .,The managers saw the student .
2,1,The presidents encouraged the banker .,The banker encouraged the presidents .
3,1,The senators supported the actor .,The actor supported the senators .
4,1,The actors avoided the bankers .,The bankers avoided the actors .
...,...,...,...
29995,0,Obviously the senator avoided the professor .,The senator avoided the professor .
29996,0,Certainly the senators recognized the actor .,The senators recognized the actor .
29997,0,Obviously the artists ran .,The artists ran .
29998,0,Suddenly the tourists supported the judges .,The tourists supported the judges .


# MNLI
moved to a different notebook because it makes use of huggingface

# HELP
https://github.com/verypluming/HELP
help dataset, data augmentation, monotonicity examples.

In [ ]:
!git clone https://github.com/verypluming/HELP.git


Cloning into 'HELP'...
remote: Enumerating objects: 68, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 68 (delta 6), reused 1 (delta 0), pack-reused 50
Unpacking objects: 100% (68/68), 892.84 KiB | 1.57 MiB/s, done.


In [ ]:
%cd HELP/output_en

/content/HELP/output_en


In [ ]:
df_help = pd.read_csv('pmb_train_v1.0.tsv', sep='\t', on_bad_lines='skip')#1
df_help['sentence1']= df_help['ori_sentence']
df_help['sentence2']= df_help['new_sentence']

In [ ]:
df_help = pd.read_pickle('HELP_train.pickle')#1
df_help

,Unnamed: 0,filename,determiner,monotonicity,gold_label,replace_target,replace_source,replace_mode,ori_sentence,new_sentence,...,text_sentence1,text_sentence2,pos_sentence1,pos_sentence2,heads_sentence1,heads_sentence2,deprel_sentence1,deprel_sentence2,sentence1_parse,sentence2_parse
0,0,silver/p75/d3454,neither,downward_monotone,entailment,neither one of his parents,neither parents,simple,Tom said that neither parents had ever been to...,Tom said that neither one of his parents had e...,...,"[Tom, said, that, neither, parents, had, ever,...","[Tom, said, that, neither, one, of, his, paren...","[NNP, VBD, IN, DT, NNS, VBD, RB, VBN, IN, NNP, .]","[NNP, VBD, IN, DT, CD, IN, PRP$, NNS, VBD, RB,...","[2, 0, 10, 5, 10, 10, 10, 10, 10, 2, 2]","[2, 0, 13, 5, 13, 8, 8, 5, 13, 13, 13, 13, 2, 2]","[nsubj, root, mark, det, nsubj, aux, advmod, c...","[nsubj, root, mark, det, nsubj, case, nmod:pos...",(ROOT (S (NP (NNP Tom)) (VP (VBD said) (SBAR (...,(ROOT (S (NP (NNP Tom)) (VP (VBD said) (SBAR (...
1,1,silver/p14/d0959,neither,downward_monotone,entailment,neither of them,neither them,simple,I know neither them .,I know neither of them .,...,"[I, know, neither, them, .]","[I, know, neither, of, them, .]","[PRP, VBP, CC, PRP, .]","[PRP, VBP, DT, IN, PRP, .]","[2, 0, 4, 2, 2]","[2, 0, 2, 5, 3, 2]","[nsubj, root, det:predet, obj, punct]","[nsubj, root, obj, case, nmod, punct]",(ROOT (S (NP (PRP I)) (VP (VBP know) (NP (CC n...,(ROOT (S (NP (PRP I)) (VP (VBP know) (NP (NP (...
2,2,silver/p24/d2663,neither,downward_monotone,entailment,neither of his brothers,neither brothers,simple,I know neither brothers .,I know neither of his brothers .,...,"[I, know, neither, brothers, .]","[I, know, neither, of, his, brothers, .]","[PRP, VBP, DT, NNS, .]","[PRP, VBP, DT, IN, PRP$, NNS, .]","[2, 0, 4, 2, 2]","[2, 0, 2, 6, 6, 3, 2]","[nsubj, root, det, obj, punct]","[nsubj, root, obj, case, nmod:poss, nmod, punct]",(ROOT (S (NP (PRP I)) (VP (VBP know) (NP (DT n...,(ROOT (S (NP (PRP I)) (VP (VBP know) (NP (NP (...
3,3,silver/p24/d2663,neither,downward_monotone,neutral,brothers,friends,noun_hypernym_obj,I know neither of his brothers .,I know neither of his friends .,...,"[I, know, neither, of, his, brothers, .]","[I, know, neither, of, his, friends, .]","[PRP, VBP, DT, IN, PRP$, NNS, .]","[PRP, VBP, DT, IN, PRP$, NNS, .]","[2, 0, 2, 6, 6, 3, 2]","[2, 0, 2, 6, 6, 3, 2]","[nsubj, root, obj, case, nmod:poss, nmod, punct]","[nsubj, root, obj, case, nmod:poss, nmod, punct]",(ROOT (S (NP (PRP I)) (VP (VBP know) (NP (NP (...,(ROOT (S (NP (PRP I)) (VP (VBP know) (NP (NP (...
4,4,silver/p24/d2663,neither,downward_monotone,entailment,brothers,friends,noun_hypernym_obj,I know neither of his friends .,I know neither of his brothers .,...,"[I, know, neither, of, his, friends, .]","[I, know, neither, of, his, brothers, .]","[PRP, VBP, DT, IN, PRP$, NNS, .]","[PRP, VBP, DT, IN, PRP$, NNS, .]","[2, 0, 2, 6, 6, 3, 2]","[2, 0, 2, 6, 6, 3, 2]","[nsubj, root, obj, case, nmod:poss, nmod, punct]","[nsubj, root, obj, case, nmod:poss, nmod, punct]",(ROOT (S (NP (PRP I)) (VP (VBP know) (NP (NP (...,(ROOT (S (NP (PRP I)) (VP (VBP know) (NP (NP (...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35886,431,silver/p97/d2664,or,disjunction,entailment,NaN,NaN,NaN,There were too many sights to see in a day,There were too many sights to see in a day or ...,...,"[There, were, too, many, sights, to, see, in, ...","[There, were, too, many, sights, to, see, in, ...","[EX, VBD, RB, JJ, NNS, TO, VB, IN, DT, NN]","[EX, VBD, RB, JJ, NNS, TO, VB, IN, DT, NN, CC,...","[2, 0, 4, 5, 2, 7, 5, 10, 10, 7]","[2, 0, 4, 5, 2, 7, 5, 10, 10, 7, 12, 10, 2]","[expl, root, advmod, amod, nsubj, mark, acl, c...","[expl, root, advmod, amod, nsubj, mark, acl, c...",(ROOT (S (NP (EX There)) (VP (VBD were) (NP (N...,(ROOT (S (NP (EX There)) (VP (VBD were) (NP (N...
35887,432,silver/p97/d2794,or,disjunction,neutral,NaN,NaN,NaN,There are two or three pens on the desk .,There are two

# SICK
https://alt.qcri.org/semeval2014/task1/index.php?id=data-and-tools#

In [ ]:
import pandas as pd

# Read in the data
df_sick_train_original = pd.read_csv('SICK_train.txt', sep='\t', on_bad_lines='skip')
df_sick_test_original = pd.read_csv('SICK_test_annotated.txt', sep='\t', on_bad_lines='skip')

# Create new dataframes
df_sick_train = pd.DataFrame({'gold_label': df_sick_train_original['entailment_judgment'].str.lower(),
                              'sentence1': df_sick_train_original['sentence_A'],
                              'sentence2': df_sick_train_original['sentence_B']})
df_sick_test = pd.DataFrame({'gold_label': df_sick_test_original['entailment_judgment'].str.lower(),
                             'sentence1': df_sick_test_original['sentence_A'],
                             'sentence2': df_sick_test_original['sentence_B']})
df_sick_train

FileNotFoundError: ignored

In [ ]:
df=df_sick_train_original
df=df[df['entailment_judgment'] == 'CONTRADICTION']
df['relatedness_score'].mean()

3.6030902255639097

In [ ]:
# sick from huggingface
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("sick", "validation")


# FeverNLI

In [ ]:
with ZipFile('nli_fever.zip', 'r') as zipfile:
    zipfile.extractall()
print('Extraction complete.')


Extraction complete.


In [ ]:
fever_train = pd.read_json('nli_fever/train_fitems.jsonl', lines=True)
fever_train = fever_train[['query','context','label']] #extract necessary columns
fever_train.columns=['sentence1', 'sentence2', 'gold_label']

In [ ]:
# convert non-ascii into ascii
!pip install unidecode
from unidecode import unidecode
fever_train['sentence1_decoded'] = fever_train['sentence1'].apply(unidecode)
fever_train['sentence2_decoded'] = fever_train['sentence2'].apply(unidecode)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 5.6 MB/s eta 0:00:00


In [ ]:
fever_train

,sentence1,sentence2,gold_label,sentence1_decoded,sentence2_decoded
0,Nikolaj Coster-Waldau worked with the Fox Broa...,The Fox Broadcasting Company ( often shortened...,SUPPORTS,Nikolaj Coster-Waldau worked with the Fox Broa...,The Fox Broadcasting Company ( often shortened...
1,Roman Atwood is a content creator.,Roman Atwood . He is best known for his vlogs ...,SUPPORTS,Roman Atwood is a content creator.,Roman Atwood . He is best known for his vlogs ...
2,Roman Atwood is a content creator.,"Roman Atwood . His vlogging channel , `` Roman...",SUPPORTS,Roman Atwood is a content creator.,"Roman Atwood . His vlogging channel , `` Roman..."
3,"History of art includes architecture, dance, s...",History of art . The subsequent expansion of t...,SUPPORTS,"History of art includes architecture, dance, s...",History of art . The subsequent expansion of t...
4,Adrienne Bailon is an accountant.,Adrienne Eliza Houghton ( née Bailon ; born O...,REFUTES,Adrienne Bailon is an accountant.,Adrienne Eliza Houghton ( nee Bailon ; born Oc...
...,...,...,...,...,...
208341,J. R. R. Tolkien created Gimli.,Gimli is a fictional character from J. R. R. T...,SUPPORTS,J. R. R. Tolkien created Gimli.,Gimli is a fictional character from J. R. R. T...
208342,J. R. R. Tolkien created Gimli.,Gimli is a fictional character from J. R. R. T...,SUPPORTS,J. R. R. Tolkien created Gimli.,Gimli is a fictional character from J. R. R. T...
208343,Susan Sarandon is an award winner.,Susan Sarandon . She was appointed a UNICEF Go...,SUPPORTS,Susan Sarandon is an award winner.,Susan Sarandon . She was appointed a UNICEF Go...
208344,Susan Sarandon is an award winner.,Susan Sarandon . She was appointed a UNICEF Go...,SUPPORTS,Susan Sarandon is an award winner.,Susan Sarandon . She was appointed a UNICEF Go...


In [ ]:
# unequal_rows = fever_train[fever_train['sentence1'] != fever_train['sentence1_decoded']]
# unequal_rows
unequal_rows = fever_train[(fever_train['sentence1'] != fever_train['sentence1_decoded']) | (fever_train['sentence2'] != fever_train['sentence2_decoded'])]
unequal_rows

,sentence1,sentence2,gold_label,sentence1_decoded,sentence2_decoded
2,Roman Atwood is a content creator.,"Roman Atwood . His vlogging channel , `` Roman...",SUPPORTS,Roman Atwood is a content creator.,"Roman Atwood . His vlogging channel , `` Roman..."
4,Adrienne Bailon is an accountant.,Adrienne Eliza Houghton ( née Bailon ; born O...,REFUTES,Adrienne Bailon is an accountant.,Adrienne Eliza Houghton ( nee Bailon ; born Oc...
6,Homeland is an American television spy thrille...,Homeland is an American spy thriller televisio...,SUPPORTS,Homeland is an American television spy thrille...,Homeland is an American spy thriller televisio...
18,Ryan Gosling has been to a country in Africa.,"Chad ( [ tʃæd ] تشاد ; Tchad [ tʃa ( d ) ] ) ,...",SUPPORTS,Ryan Gosling has been to a country in Africa.,Chad ( [ tSaed ] tshd ; Tchad [ tSa ( d ) ] ) ...
28,Selena recorded music.,Selena Quintanilla-Pérez ( [ seˈlena kintaˈni...,SUPPORTS,Selena recorded music.,Selena Quintanilla-Perez ( [ se'lena kinta'nij...
...,...,...,...,...,...
208325,Kenneth Branagh is a follower.,Sir Kenneth Charles Branagh ( [ ˈbrænə ] born ...,NOT ENOUGH INFO,Kenneth Branagh is a follower.,Sir Kenneth Charles Branagh ( [ 'braen@ ] born...
208329,Jeff Goldblum starred in a film.,Jeffrey Lynn Goldblum ( [ ˈɡoʊldbluːm ] born O...,SUPPORTS,Jeff Goldblum starred in a film.,Jeffrey Lynn Goldblum ( [ 'goUldblu:m ] born O...
208330,Jeff Goldblum starred in a film.,Cats & Dogs is a 2001 American-Australian fami...,SUPPORTS,Jeff Goldblum starred in a film.,Cats & Dogs is a 2001 American-Australian fami...
208340,Her stars American actress Rooney Mara.,"Her (film) . The film also stars Amy Adams , R...",SUPPORTS,Her stars American actress Rooney Mara.,"Her (film) . The film also stars Amy Adams , R..."


In [ ]:
fever_train


,sentence1,sentence2,gold_label
0,Nikolaj Coster-Waldau worked with the Fox Broa...,The Fox Broadcasting Company ( often shortened...,SUPPORTS
1,Roman Atwood is a content creator.,Roman Atwood . He is best known for his vlogs ...,SUPPORTS
2,Roman Atwood is a content creator.,"Roman Atwood . His vlogging channel , `` Roman...",SUPPORTS
3,"History of art includes architecture, dance, s...",History of art . The subsequent expansion of t...,SUPPORTS
4,Adrienne Bailon is an accountant.,Adrienne Eliza Houghton ( nee Bailon ; born Oc...,REFUTES
...,...,...,...
208341,J. R. R. Tolkien created Gimli.,Gimli is a fictional character from J. R. R. T...,SUPPORTS
208342,J. R. R. Tolkien created Gimli.,Gimli is a fictional character from J. R. R. T...,SUPPORTS
208343,Susan Sarandon is an award winner.,Susan Sarandon . She was appointed a UNICEF Go...,SUPPORTS
208344,Susan Sarandon is an award winner.,Susan Sarandon . She was appointed a UNICEF Go...,SUPPORTS


In [ ]:
# convert gold label
fever_train['gold_label'] = fever_train['gold_label'].replace('SUPPORTS', 'entailment')
fever_train['gold_label'] = fever_train['gold_label'].replace('REFUTES', 'contradiction')
fever_train['gold_label'] = fever_train['gold_label'].replace('NOT ENOUGH INFO', 'neutral')
fever_train['gold_label'].unique()

array(['entailment', 'contradiction', 'neutral'], dtype=object)

In [ ]:
len(fever_train)
#208346

208346

In [ ]:
fever_train.loc[fever_train['sentence2'].str.strip().str.len() == 0]

,sentence1,sentence2,gold_label
44,Marilyn Monroe worked with Warner Brothers.,,neutral
53,United Nations was relocated to the United Sta...,,neutral
92,Richard Eyre was born on an island.,,neutral
141,Nicole Scherzinger toured 15 times.,,neutral
143,Sade (singer) has a song called London.,,neutral
...,...,...,...
208061,K-pop Star 2 is a sequel to K-pop Star 1.,,neutral
208133,Chris Pine appeared at a comic convention.,,neutral
208179,Judi Dench played a board game.,,neutral
208228,Mexico's economy includes sharecropping.,,neutral


In [ ]:
# replace whitespace cols with n/a
fever_train.loc[fever_train['sentence2'].str.strip().str.len() == 0, 'sentence2'] = 'n/a'
# df = fever_train[fever_train['sentence2'].str.strip() == 'n/a']

In [ ]:
# split fever dataset into 3 equal parts
n = len(fever_train)

# split into three parts
fever_train1 = fever_train.iloc[:n//3]
fever_train2 = fever_train.iloc[n//3:(2*n)//3]
fever_train3 = fever_train.iloc[(2*n)//3:]

# Make sure the lengths add up
assert len(fever_train) == len(fever_train1) + len(fever_train2) + len(fever_train3)


## fever test

In [ ]:
# test set
# fever_test = pd.read_json('nli_fever/test_fitems.jsonl', lines=True)
# fever_test = fever_test[['query','context','label']] #extract necessary columns
# fever_test.columns=['sentence1', 'sentence2', 'gold_label']
# # convert gold label
# fever_test['gold_label'] = fever_test['gold_label'].replace('SUPPORTS', 'entailment')
# fever_test['gold_label'] = fever_test['gold_label'].replace('REFUTES', 'contradiction')
# fever_test['gold_label'] = fever_test['gold_label'].replace('NOT ENOUGH INFO', 'neutral')
# fever_test['gold_label'].unique()
# # replace whitespace cols with n/a
# fever_test.loc[fever_test['sentence2'].str.strip().str.len() == 0, 'sentence2'] = 'n/a'

array(['hidden'], dtype=object)

In [ ]:
# len(fever_test)

19998

# 2.Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


# 3.config + run below

In [ ]:
output_file_name= 'fever_train1'
df_to_save = fever_train1 #<- change to the df u want to save, then run below
zip=True
save_to_drive=True

In [ ]:
nlp = stanza.Pipeline('en', processors='tokenize,mwt,pos,lemma,depparse,constituency', use_gpu=True, tokenize_no_ssplit=True)

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: en (English):
| Processor    | Package  |
---------------------------
| tokenize     | combined |
| pos          | combined |
| lemma        | combined |
| constituency | wsj      |
| depparse     | combined |

INFO:stanza:Using device: cuda
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: constituency
INFO:stanza:Loading: depparse
INFO:stanza:Done loading processors!


In [ ]:
# #dep parsing
# sentences1=list(df_to_save['sentence1'])
# sentences2=list(df_to_save['sentence2'])

# start = time.time()
# in_docs = [stanza.Document([], text=d) for d in sentences1] # Wrap each document with a stanza.Document object
# docs1 = nlp(in_docs) # Call the neural pipeline on this list of documents

# mid = time.time()
# print(f'execution time docs1: {mid-start}')

# in_docs = [stanza.Document([], text=d) for d in sentences2] # Wrap each document with a stanza.Document object
# docs2 = nlp(in_docs) # Call the neural pipeline on this list of documents

# end = time.time()
# print(f'execution time docs2: {end-mid}')
# print(f'execution time total: {end-start}')


In [ ]:
#added truncation
import tqdm

# Dependency parsing
sentences1 = list(df_to_save['sentence1'])
sentences2 = list(df_to_save['sentence2'])

# Maximum sequence length
max_length = 100

# Truncate a sentence string to the specified length
#
def truncate_sentence(sentence, max_length):
    words = sentence.split()
    #ascii_words = [word for word in words if word.isascii()]
    truncated_words = words[:max_length]
    return ' '.join(truncated_words)

# Wrap the truncated sentences with stanza.Document objects
in_docs1 = [stanza.Document([], text=truncate_sentence(d, max_length)) for d in sentences1]
in_docs2 = [stanza.Document([], text=truncate_sentence(d, max_length)) for d in sentences2]

# Set batch size
batch_size = 64

# Process the sentences in batches using the Stanza pipeline
def process_batches(in_docs):
    processed_docs = []
    for i in tqdm.tqdm(range(0, len(in_docs), batch_size), desc="Processing batches"):
        batch = in_docs[i:i + batch_size]
        processed_batch = nlp(batch)
        processed_docs.extend(processed_batch)
    return processed_docs

docs1 = process_batches(in_docs1)
docs2 = process_batches(in_docs2)


Processing batches: 100%|██████████| 1086/1086 [1:09:53<00:00,  3.86s/it]


In [ ]:
# text1, text2 = [],[]
# postags1, postags2 = [],[]
# heads1, heads2 = [],[]
# deprel1, deprel2 =[],[]
# const1,const2=[],[]

# for sentence in docs1:
#   text1.append([x['text'] for x in sentence.sentences[0].to_dict()])
#   postags1.append([x['xpos'] for x in sentence.sentences[0].to_dict()])
#   heads1.append([x['head'] for x in sentence.sentences[0].to_dict()])
#   deprel1.append([x['deprel'] for x in sentence.sentences[0].to_dict()])
#   const1.append(str(sentence.sentences[0].constituency))

# for sentence in docs2:
#   text2.append([x['text'] for x in sentence.sentences[0].to_dict()])
#   postags2.append([x['xpos'] for x in sentence.sentences[0].to_dict()])
#   heads2.append([x['head'] for x in sentence.sentences[0].to_dict()])
#   deprel2.append([x['deprel'] for x in sentence.sentences[0].to_dict()])
#   const2.append(str(sentence.sentences[0].constituency))

# assert len(text1)==len(deprel1) == len(heads1) == len(postags1) == len(const1)== len(df_to_save)
# assert len(text2)==len(deprel2) == len(heads2) ==len(postags2) ==len(const2)== len(df_to_save)
def extract_data(docs):
    text, postags, heads, deprel, const = [], [], [], [], []
    for i, sentence in enumerate(docs):
        # sentence_dict = sentence.sentences[0].to_dict()
        # text.append([x['text'] for x in sentence_dict])
        # postags.append([x['xpos'] for x in sentence_dict])
        # heads.append([x['head'] for x in sentence_dict])
        # deprel.append([x['deprel'] for x in sentence_dict])
        # const.append(str(sentence.sentences[0].constituency))
        if len(sentence.sentences)>0:
            sentence_dict = sentence.sentences[0].to_dict()
            text.append([x['text'] for x in sentence_dict])
            postags.append([x['xpos'] for x in sentence_dict])
            heads.append([x['head'] for x in sentence_dict])
            deprel.append([x['deprel'] for x in sentence_dict])
            const.append(str(sentence.sentences[0].constituency))
        else:
            text.append([])
            postags.append([])
            heads.append([])
            deprel.append([])
            const.append('')
    return text, postags, heads, deprel, const

text1, postags1, heads1, deprel1, const1 = extract_data(docs1)
text2, postags2, heads2, deprel2, const2 = extract_data(docs2)

assert (len(text1) == len(deprel1) == len(heads1) == len(postags1) == len(const1) == len(df_to_save))
assert (len(text2) == len(deprel2) == len(heads2) == len(postags2) == len(const2) == len(df_to_save))



In [ ]:

df_to_save = df_to_save.assign(text_sentence1=text1,
                               text_sentence2=text2,
                               pos_sentence1=postags1,
                               pos_sentence2=postags2,
                               heads_sentence1=heads1,
                               heads_sentence2=heads2,
                               deprel_sentence1=deprel1,
                               deprel_sentence2=deprel2,
                               sentence1_parse=const1,
                               sentence2_parse=const2)


In [ ]:
df_to_save

,sentence1,sentence2,gold_label,text_sentence1,text_sentence2,pos_sentence1,pos_sentence2,heads_sentence1,heads_sentence2,deprel_sentence1,deprel_sentence2,sentence1_parse,sentence2_parse
0,Nikolaj Coster-Waldau worked with the Fox Broa...,The Fox Broadcasting Company ( often shortened...,entailment,"[Nikolaj, Coster, -, Waldau, worked, with, the...","[The, Fox, Broadcasting, Company, (, often, sh...","[NNP, NNP, HYPH, NNP, VBD, IN, DT, NNP, NNP, N...","[DT, NNP, NNP, NNP, -LRB-, RB, VBN, IN, NNP, C...","[5, 1, 4, 1, 0, 10, 10, 10, 10, 5, 5]","[4, 4, 4, 23, 7, 7, 4, 9, 7, 11, 7, 13, 11, 7,...","[nsubj, flat, punct, flat, root, case, det, co...","[det, compound, compound, nsubj, punct, advmod...",(ROOT (S (NP (NNP Nikolaj) (NNP Coster) (HYPH ...,(ROOT (S (S (NP (NP (DT The) (NNP Fox) (NNP Br...
1,Roman Atwood is a content creator.,Roman Atwood . He is best known for his vlogs ...,entailment,"[Roman, Atwood, is, a, content, creator, .]","[Roman, Atwood, ., He, is, best, known, for, h...","[NNP, NNP, VBZ, DT, NN, NN, .]","[NNP, NNP, ., PRP, VBZ, RBS, VBN, IN, PRP$, NN...","[6, 1, 6, 6, 6, 0, 6]","[7, 1, 7, 7, 7, 7, 0, 10, 10, 7, 14, 14, 14, 1...","[nsubj, flat, cop, det, compound, root, punct]","[vocative, flat, punct, nsubj:pass, aux:pass, ...",(ROOT (S (NP (NNP Roman) (NNP Atwood)) (VP (VB...,(ROOT (S (S (NP (NNP Roman) (NNP Atwood) (. .)...
2,Roman Atwood is a content creator.,"Roman Atwood . His vlogging channel , `` Roman...",entailment,"[Roman, Atwood, is, a, content, creator, .]","[Roman, Atwood, ., His, vlogging, channel, ,, ...","[NNP, NNP, VBZ, DT, NN, NN, .]","[NNP, NNP, ., PRP$, NN, NN, ,, ``, NNP, '', ,,...","[6, 1, 6, 6, 6, 0, 6]","[12, 1, 1, 6, 6, 12, 9, 9, 6, 9, 12, 0, 14, 12...","[nsubj, flat, cop, det, compound, root, punct]","[nsubj, flat, punct, nmod:poss, compound, nsub...",(ROOT (S (NP (NNP Roman) (NNP Atwood)) (VP (VB...,(ROOT (S (NP (NNP Roman) (NNP Atwood) (. .)) (...
3,"History of art includes architecture, dance, s...",History of art . The subsequent expansion of t...,entailment,"[History, of, art, includes, architecture, ,, ...","[History, of, art, ., The, subsequent, expansi...","[NN, IN, NN, VBZ, NN, ,, NN, ,, NN, ,, NN, ,, ...","[NN, IN, NN, ., DT, JJ, NN, IN, DT, NN, IN, JJ...","[4, 3, 1, 0, 4, 7, 5, 9, 5, 11, 5, 13, 5, 16, ...","[0, 3, 1, 1, 7, 7, 18, 10, 10, 7, 13, 13, 10, ...","[nsubj, case, nmod, root, obj, punct, conj, pu...","[root, case, nmod, punct, det, amod, nsubj, ca...",(ROOT (S (NP (NP (NN History)) (PP (IN of) (NP...,(ROOT (S (NP (NP (NP (NN History)) (PP (IN of)...
4,Adrienne Bailon is an accountant.,Adrienne Eliza Houghton ( nee Bailon ; born Oc...,contradiction,"[Adrienne, Bailon, is, an, accountant, .]","[Adrienne, Eliza, Houghton, (, nee, Bailon, ;,...","[NNP, NNP, VBZ, DT, NN, .]","[NNP, NNP, NNP, -LRB-, NNP, NNP, :, VBN, NNP, ...","[5, 1, 5, 5, 0, 5]","[19, 1, 1, 5, 1, 5, 8, 1, 10, 8, 12, 10, 5, 19...","[nsubj, flat, cop, det, root, punct]","[nsubj, flat, flat, punct, parataxis, flat, pu...",(ROOT (S (NP (NNP Adrienne) (NNP Bailon)) (VP ...,(ROOT (S (NP (NP (NNP Adrienne) (NNP Eliza) (N...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69443,Kevin Durant played one season of college lacr...,Kevin Durant . He played one season of college...,contradiction,"[Kevin, Durant, played, one, season, of, colle...","[Kevin, Durant, ., He, played, one, season, of...","[NNP, NNP, VBD, CD, NN, IN, NN, NN, IN, DT, NN...","[NNP, NNP, ., PRP, VBD, CD, NN, IN, NN, NN, IN...","[3, 1, 0, 5, 3, 8, 8, 5, 11, 11, 3, 13, 11, 3]","[0, 1, 1, 5, 1, 7, 5, 10, 10, 7, 13, 13, 5, 15...","[nsubj, flat, root, nummod, obj, case, compoun...","[root, flat, punct, nsubj, parataxis, nummod, ...",(ROOT (S (NP (NNP Kevin) (NNP Durant)) (VP (VB...,(ROOT (NP (NP (NNP Kevin) (NNP Durant)) (. .) ...
69444,There Will Be Blood has been released.,There Will Be Blood is a 2007 American epic hi...,entailment,"[There, Will, Be, Blood, has, been, released, .]","[There, Will, Be, Blood, is, a, 2007, American...","[EX, M

In [ ]:
# df_test = df_to_save[df_to_save['text_sentence2'].str.len() ==0]
# df_test

## pickle the data, zip, save to drive

In [ ]:
#save data
df_to_save.to_pickle(output_file_name+'.pickle')

In [ ]:

f = f"{output_file_name+'.zip'}"
out = output_file_name + '.pickle'
#!zip -r '/content/folder/"NewFileName.zip"' '/content/folder/"OrginalFileName"'

if zip:
  !zip -r '{output_file_name+'.zip'}' '{output_file_name+'.pickle'}'


  adding: fever_train1.pickle (deflated 80%)


In [ ]:
if zip and save_to_drive:
  !cp -r /content/{output_file_name+".zip"} /content/gdrive/MyDrive

cp: cannot create regular file '/content/gdrive/MyDrive': No such file or directory


In [ ]:
# Save data
df_to_save.to_pickle(f"{output_file_name}.pickle")

# Zip and save
if zip:
    !zip -r "{output_file_name}.zip" "{output_file_name}.pickle"
    if save_to_drive:
        !cp -r /content/{output_file_name}.zip /content/gdrive/MyDrive


updating: fever_train1.pickle (deflated 80%)
